<a href="https://colab.research.google.com/github/suinkangme/comp433_project/blob/main/Transfer_Learning_SK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing a robust CNN model to address the challenge of learning with label noise in  CIFAR10 dataset

- CIFAR10 Label : ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’.

- image size : 3x32x32




In [24]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, random_split
import os
from torchvision.models import resnet18

## Load and normalize CIFAR10

- Transform them to Tensors of normalized range [-1, 1].

In [14]:
# train dataset
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])
cifar10_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)

Files already downloaded and verified


In [15]:
# validation, test dataset
test_val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [16]:
# split the dataset into train and validation
train_size = int(0.8 * len(cifar10_dataset))
val_size = len(cifar10_dataset) - train_size
train_dataset, val_dataset = random_split(cifar10_dataset, [train_size, val_size])

In [17]:
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = test_val_transform

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [18]:
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_val_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified


## Noise Labeling
- 5 different noise levels (10%,
30%, 50%, 80%, 90%)

In [19]:
def apply_label_noise(labels, epsilon, noise_type):
    num_labels = len(labels)
    num_flips = int(epsilon * num_labels)

    if noise_type == 'symmetric':
        # Symmetric label noise
        flip_indices = np.random.choice(num_labels, num_flips, replace=False)
        labels[flip_indices] = np.random.randint(0, 10, num_flips)
    elif noise_type == 'asymmetric':
        # Asymmetric label noise
        flip_rules = {
            9: 1,   # Truck to Automobile
            2: 0,   # Bird to Airplane
            4: 7,   # Deer to Horse
            3: 5,   # Cat to Dog
            5: 3,   # Dog to Cat
        }

        for i in range(num_labels):
            if np.random.random() < epsilon:
                labels[i] = flip_rules.get(labels[i], labels[i])

    return labels

# **Baseline Model**

## Define a  baseline CNN model

In [30]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # fc layers
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 8 * 8, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.fc_layers(x)
        return x

## Train & Validation and Testing - symmetric noise label

In [21]:
# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

# create a dictionary with keys in the format 'noise_level_{100 * value}'
model_dict = {f'noise_level_{int(100 * level)}_sy': None for level in noise_levels}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# training
for epsilon in noise_levels:

    num_epochs = 5

    model = BaselineModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model = model.to(device)
    model.train()

    print(f"Symmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:

            # add symmetric noise to labels
            labels = apply_label_noise(labels.numpy(), epsilon=epsilon, noise_type='symmetric')
            labels = torch.from_numpy(labels)

            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

    # validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    average_val_loss = val_loss / len(val_loader)

    print(f'Validation Loss: {average_val_loss}, Validation Accuracy: {val_accuracy}')

    # save model to dictionary
    model_dict[f'noise_level_{int(100 * epsilon)}_sy'] = {
        'state_dict': model.state_dict(),
        'validation_loss': average_val_loss,
        'validation_accuracy': val_accuracy
    }

Symmetric Training with noise level: 0.1
Epoch 1/5, Loss: 1.574057936668396
Epoch 2/5, Loss: 1.313593864440918
Epoch 3/5, Loss: 1.4688284397125244
Epoch 4/5, Loss: 0.9660269021987915
Epoch 5/5, Loss: 1.2292925119400024
Validation Loss: 0.9072915968621612, Validation Accuracy: 0.6976
Symmetric Training with noise level: 0.3
Epoch 1/5, Loss: 1.8488606214523315
Epoch 2/5, Loss: 1.9562700986862183
Epoch 3/5, Loss: 1.7214739322662354
Epoch 4/5, Loss: 1.7651171684265137
Epoch 5/5, Loss: 1.7388719320297241
Validation Loss: 1.1064094020302888, Validation Accuracy: 0.6697
Symmetric Training with noise level: 0.5
Epoch 1/5, Loss: 2.238291025161743
Epoch 2/5, Loss: 2.085500955581665
Epoch 3/5, Loss: 1.895177960395813
Epoch 4/5, Loss: 1.9388082027435303
Epoch 5/5, Loss: 2.0612411499023438
Validation Loss: 1.4341619804406622, Validation Accuracy: 0.6197
Symmetric Training with noise level: 0.8
Epoch 1/5, Loss: 2.3039259910583496
Epoch 2/5, Loss: 2.3062257766723633
Epoch 3/5, Loss: 2.281157016754150

In [22]:
# testing
for key, model_state_info in model_dict.items():

    model = BaselineModel()
    model_state = model.state_dict()
    model_state.update({k: v for k, v in model_state_info['state_dict'].items() if k in model_state})

    # load the updated state_dict
    model.load_state_dict(model_state)

    model = model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy for {key}: {accuracy}')

Test Accuracy for noise_level_10_sy: 0.6976
Test Accuracy for noise_level_30_sy: 0.6697
Test Accuracy for noise_level_50_sy: 0.6197
Test Accuracy for noise_level_80_sy: 0.3304
Test Accuracy for noise_level_90_sy: 0.1021


## Train & Validation and Testing - asymmetric noise label

In [32]:
# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

# create a dictionary with keys in the format 'noise_level_{100 * value}'
model_dict = {f'noise_level_{int(100 * level)}_asy': None for level in noise_levels}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# training
for epsilon in noise_levels:

    num_epochs = 5

    model = BaselineModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model = model.to(device)
    model.train()

    print(f"Asymmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:

            # add symmetric noise to labels
            labels = apply_label_noise(labels.numpy(), epsilon=epsilon, noise_type='asymmetric')
            labels = torch.from_numpy(labels)

            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

    # validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    average_val_loss = val_loss / len(val_loader)

    print(f'Validation Loss: {average_val_loss}, Validation Accuracy: {val_accuracy}')

    # save model to dictionary
    model_dict[f'noise_level_{int(100 * epsilon)}_asy'] = {
        'state_dict': model.state_dict(),
        'validation_loss': average_val_loss,
        'validation_accuracy': val_accuracy
    }

Asymmetric Training with noise level: 0.1
Epoch 1/5, Loss: 1.2843244075775146
Epoch 2/5, Loss: 1.099171757698059
Epoch 3/5, Loss: 1.091235637664795
Epoch 4/5, Loss: 0.7557635307312012
Epoch 5/5, Loss: 0.8707022666931152
Validation Loss: 0.8434161699501572, Validation Accuracy: 0.7085
Asymmetric Training with noise level: 0.3
Epoch 1/5, Loss: 1.4024876356124878
Epoch 2/5, Loss: 1.2790480852127075
Epoch 3/5, Loss: 1.1926416158676147
Epoch 4/5, Loss: 0.9048477411270142
Epoch 5/5, Loss: 0.8954256772994995
Validation Loss: 0.9617564602262655, Validation Accuracy: 0.679
Asymmetric Training with noise level: 0.5
Epoch 1/5, Loss: 1.2890602350234985
Epoch 2/5, Loss: 1.0025088787078857
Epoch 3/5, Loss: 0.9534822702407837
Epoch 4/5, Loss: 0.8951865434646606
Epoch 5/5, Loss: 1.0448288917541504
Validation Loss: 1.0258914528379015, Validation Accuracy: 0.5956
Asymmetric Training with noise level: 0.8
Epoch 1/5, Loss: 1.4586886167526245
Epoch 2/5, Loss: 1.0877671241760254
Epoch 3/5, Loss: 1.258889555

In [34]:
# testing
for key, model_state_info in model_dict.items():

    model = BaselineModel()
    model_state = model.state_dict()
    model_state.update({k: v for k, v in model_state_info['state_dict'].items() if k in model_state})

    # load the updated state_dict
    model.load_state_dict(model_state)

    model = model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy for {key}: {accuracy}')

Test Accuracy for noise_level_10_asy: 0.7121
Test Accuracy for noise_level_30_asy: 0.6732
Test Accuracy for noise_level_50_asy: 0.5824
Test Accuracy for noise_level_80_asy: 0.4588
Test Accuracy for noise_level_90_asy: 0.4501


# Transfer Learning
- Used pre-trained Resnet-18 by ImageNet

## Train & Validation and Testing - symmetric noise label

In [26]:
# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]
num_classes = 10

# create a dictionary with keys in the format 'noise_level_{100 * value}'
model_dict = {f'noise_level_{int(100 * level)}_sy': None for level in noise_levels}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# training
for epsilon in noise_levels:
    num_epochs = 5

    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model = model.to(device)
    model.train()

    print(f"Symmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:

            # add symmetric noise to labels
            labels = apply_label_noise(labels.numpy(), epsilon=epsilon, noise_type='symmetric')
            labels = torch.from_numpy(labels)

            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

    # validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    average_val_loss = val_loss / len(val_loader)

    print(f'Validation Loss: {average_val_loss}, Validation Accuracy: {val_accuracy}')

    # save model to dictionary
    model_dict[f'noise_level_{int(100 * epsilon)}_sy'] = {
        'state_dict': model.state_dict(),
        'validation_loss': average_val_loss,
        'validation_accuracy': val_accuracy
    }

Symmetric Training with noise level: 0.1
Epoch 1/5, Loss: 1.4383738040924072
Epoch 2/5, Loss: 0.8490682244300842
Epoch 3/5, Loss: 1.07673978805542
Epoch 4/5, Loss: 0.8105618953704834
Epoch 5/5, Loss: 0.8231915235519409
Validation Loss: 0.7404763877012168, Validation Accuracy: 0.7643
Symmetric Training with noise level: 0.3
Epoch 1/5, Loss: 1.8383228778839111
Epoch 2/5, Loss: 1.6291735172271729
Epoch 3/5, Loss: 1.667047381401062
Epoch 4/5, Loss: 1.4115960597991943
Epoch 5/5, Loss: 1.4737496376037598
Validation Loss: 0.9268511290762834, Validation Accuracy: 0.7708
Symmetric Training with noise level: 0.5
Epoch 1/5, Loss: 2.152388334274292
Epoch 2/5, Loss: 2.08945894241333
Epoch 3/5, Loss: 1.9389090538024902
Epoch 4/5, Loss: 2.023054361343384
Epoch 5/5, Loss: 2.1566367149353027
Validation Loss: 2.9227642565016536, Validation Accuracy: 0.4191
Symmetric Training with noise level: 0.8
Epoch 1/5, Loss: 2.263587713241577
Epoch 2/5, Loss: 2.293323278427124
Epoch 3/5, Loss: 2.3045923709869385
Ep

In [27]:
# testing
for key, model_state_info in model_dict.items():

    model = resnet18(pretrained=True)
    num_classes = 10
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    model_state = model.state_dict()
    model_state.update({k: v for k, v in model_state_info['state_dict'].items() if k in model_state})

    # load the updated state_dict
    model.load_state_dict(model_state)

    model = model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy for {key}: {accuracy}')


Test Accuracy for noise_level_10_sy: 0.7643
Test Accuracy for noise_level_30_sy: 0.7708
Test Accuracy for noise_level_50_sy: 0.4191
Test Accuracy for noise_level_80_sy: 0.2845
Test Accuracy for noise_level_90_sy: 0.1754


## Train & Validation and Testing - asymmetric noise label

In [28]:
0# noise_levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]
num_classes = 10

# create a dictionary with keys in the format 'noise_level_{100 * value}'
model_dict = {f'noise_level_{int(100 * level)}_asy': None for level in noise_levels}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# training
for epsilon in noise_levels:
    num_epochs = 5

    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model = model.to(device)
    model.train()

    print(f"Asymmetric Training with noise level: {epsilon}")

    for epoch in range(num_epochs):
        for inputs, labels in train_loader:

            # add symmetric noise to labels
            labels = apply_label_noise(labels.numpy(), epsilon=epsilon, noise_type='asymmetric')
            labels = torch.from_numpy(labels)

            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

    # validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    average_val_loss = val_loss / len(val_loader)

    print(f'Validation Loss: {average_val_loss}, Validation Accuracy: {val_accuracy}')

    # save model to dictionary
    model_dict[f'noise_level_{int(100 * epsilon)}_asy'] = {
        'state_dict': model.state_dict(),
        'validation_loss': average_val_loss,
        'validation_accuracy': val_accuracy
    }

Asymmetric Training with noise level: 0.1
Epoch 1/5, Loss: 0.7395397424697876
Epoch 2/5, Loss: 0.7459056973457336
Epoch 3/5, Loss: 0.4033662676811218
Epoch 4/5, Loss: 0.7493487000465393
Epoch 5/5, Loss: 0.4946557879447937
Validation Loss: 0.6455741072915921, Validation Accuracy: 0.7905
Asymmetric Training with noise level: 0.3
Epoch 1/5, Loss: 0.8711126446723938
Epoch 2/5, Loss: 0.8916053771972656
Epoch 3/5, Loss: 0.8082765340805054
Epoch 4/5, Loss: 0.712039589881897
Epoch 5/5, Loss: 0.5307481288909912
Validation Loss: 0.7484993021579305, Validation Accuracy: 0.7783
Asymmetric Training with noise level: 0.5
Epoch 1/5, Loss: 1.0165116786956787
Epoch 2/5, Loss: 1.0446757078170776
Epoch 3/5, Loss: 0.7705175280570984
Epoch 4/5, Loss: 0.7575512528419495
Epoch 5/5, Loss: 0.6496548056602478
Validation Loss: 0.8887693809855516, Validation Accuracy: 0.5438
Asymmetric Training with noise level: 0.8
Epoch 1/5, Loss: 0.8771190643310547
Epoch 2/5, Loss: 0.528442919254303
Epoch 3/5, Loss: 0.57564663

In [29]:
# testing
for key, model_state_info in model_dict.items():

    model = resnet18(pretrained=True)
    num_classes = 10
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    model_state = model.state_dict()
    model_state.update({k: v for k, v in model_state_info['state_dict'].items() if k in model_state})

    # load the updated state_dict
    model.load_state_dict(model_state)

    model = model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy for {key}: {accuracy}')


Test Accuracy for noise_level_10_asy: 0.7905
Test Accuracy for noise_level_30_asy: 0.7783
Test Accuracy for noise_level_50_asy: 0.5438
Test Accuracy for noise_level_80_asy: 0.4544
Test Accuracy for noise_level_90_asy: 0.4746
